In [1]:
import pandas as pd
import numpy as np

## Loading Dataset

In [2]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/fashion-mnist_train.csv')

#### Check for GPU

In [3]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Splitting

In [4]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
from sklearn.model_selection import train_test_split as split

train_x, test_x, train_y, test_y = split(df.drop(columns = ['label']).values, df['label'].values, test_size=0.2, random_state=42)

# Transformer

In [6]:
from torchvision.transforms import transforms

custom_transform = transforms.Compose([
    transforms.Resize(256), # step 5
    transforms.CenterCrop(224), # step 6
    transforms.ToTensor(), # step 7
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # step 8
])

In [7]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, features, labels, transform):
        self.features = features
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.features)

    def __getitem__(self, index): # initail steps
        # Resize
        image = self.features[index].reshape(28, 28)

        # change the data typeto np.uint8
        image = image.astype(np.uint8)

        # change black&white to color -> (h, w, c) -> (c, h, w)
        image = np.stack([image]*3, axis=-1)

        # Convert image to PLI image
        image = Image.fromarray(image)

        # Apply transform
        image = self.transform(image)

        # return transform
        return image, torch.tensor(self.labels[index], dtype=torch.long)

In [8]:
train_dataset = CustomDataset(train_x, train_y, custom_transform)
test_dataset = CustomDataset(test_x, test_y, custom_transform)

In [9]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory = True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory = True)

# VGG16 Model download

In [10]:
import torchvision.models as models
vgg16 = models.vgg16(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 106MB/s]


In [ ]:
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [11]:
for params in vgg16.features.parameters():
    params.requires_grad = False

In [18]:
import torch.nn as nn


vgg16.classifier = nn.Sequential(
    nn.Linear(25088, 1024),
    nn.ReLU(),

    nn.Dropout(0.5),

    nn.Linear(1024, 512),
    nn.ReLU(),

    nn.Dropout(0.5),

    nn.Linear(512, 10)
)

In [19]:
vgg16 = vgg16.to(device)

In [20]:
learning_rate = 0.0001
epochs = 30

In [21]:
from torch import nn, optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.classifier.parameters(), lr=learning_rate)

# Training

In [22]:
# training loop

for epoch in range(epochs):

  total_epoch_loss = 0

  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    # forward pass
    outputs = vgg16(batch_features)

    print(outputs.shape)
    print(batch_labels.shape)

    # calculate loss
    loss = criterion(outputs, batch_labels)

    # back pass
    optimizer.zero_grad()
    loss.backward()

    # update grads
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

    break

  avg_loss = total_epoch_loss/len(train_loader)
  print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')


torch.Size([32, 10])
torch.Size([32])
Epoch: 1 , Loss: 0.0015327490170796711
torch.Size([32, 10])
torch.Size([32])
Epoch: 2 , Loss: 0.0014979143142700194
torch.Size([32, 10])
torch.Size([32])
Epoch: 3 , Loss: 0.0014671386082967123
torch.Size([32, 10])
torch.Size([32])
Epoch: 4 , Loss: 0.0014344433148701986
torch.Size([32, 10])
torch.Size([32])
Epoch: 5 , Loss: 0.0013495109875996908
torch.Size([32, 10])
torch.Size([32])
Epoch: 6 , Loss: 0.0013521698315938314
torch.Size([32, 10])
torch.Size([32])
Epoch: 7 , Loss: 0.0013255581061045328
torch.Size([32, 10])
torch.Size([32])
Epoch: 8 , Loss: 0.0012944010893503825
torch.Size([32, 10])
torch.Size([32])
Epoch: 9 , Loss: 0.001298709789911906
torch.Size([32, 10])
torch.Size([32])
Epoch: 10 , Loss: 0.001189007600148519
torch.Size([32, 10])
torch.Size([32])
Epoch: 11 , Loss: 0.0011792364915211996
torch.Size([32, 10])
torch.Size([32])
Epoch: 12 , Loss: 0.0010588219960530598
torch.Size([32, 10])
torch.Size([32])
Epoch: 13 , Loss: 0.00105458887418111

# Evaluation

In [ ]:
vgg16.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# evaluation on test data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = vgg16(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)